# Waste Classification Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import json

import torch
from torch import (nn, optim)
from torch.utils.data import DataLoader
from torchvision import (transforms, datasets, models)

import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")
plt.rcParams["font.family"] = "monospace"

## Constants

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DTYPE = torch.float32
# Path for 'data'
PATH = '/content/drive/MyDrive/SAMSUNGBootcampFiles/data'   # Change for different user
EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 1e-3

## Data Preparation

In [ ]:
data = datasets.ImageFolder(
    root=PATH,
    transform=transforms.ToTensor()
)

In [ ]:
data.class_to_idx   # Classes as a dict

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}

In [ ]:
train_set, val_set = torch.utils.data.random_split(data, [.9, .1], generator=torch.Generator().manual_seed(42))
print(f"Train Set: {len(train_set)}")
print(f"Validation Set: {len(val_set)}")

Train Set: 2252
Validation Set: 250


In [ ]:
train_loader = DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_set,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## Model Creation `mobilenetv3`

In [ ]:
Model = models.mobilenet_v3_small()
Model.classifier[3] = nn.Linear(Model.classifier[3].in_features, 6)  # Adjust for 6 classes

Model = Model.to(DEVICE)

In [ ]:
optimizer = optim.Adam(Model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and Optimization
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct / total
    print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}")

    return epoch_loss, epoch_accuracy

In [ ]:
# Evaluation function
def evaluate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Statistics
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = val_loss / len(val_loader)
    epoch_accuracy = correct / total
    print(f"Validation Loss: {epoch_loss:.4f}, Validation Accuracy: {epoch_accuracy:.4f}")

    return epoch_loss, epoch_accuracy

In [ ]:
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []

for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    train_loss, train_acc = train(Model, train_loader, optimizer, criterion, DEVICE)
    val_loss, val_acc = evaluate(Model, val_loader, criterion, DEVICE)

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

Epoch 1/20
Train Loss: 1.3399, Train Accuracy: 0.4698
Validation Loss: 1.7540, Validation Accuracy: 0.2080
Epoch 2/20
Train Loss: 1.0791, Train Accuracy: 0.5955
Validation Loss: 1.8104, Validation Accuracy: 0.2000
Epoch 3/20
Train Loss: 0.9490, Train Accuracy: 0.6479
Validation Loss: 1.8075, Validation Accuracy: 0.2000
Epoch 4/20
Train Loss: 0.8196, Train Accuracy: 0.7065
Validation Loss: 2.2980, Validation Accuracy: 0.2080
Epoch 5/20
Train Loss: 0.7602, Train Accuracy: 0.7242
Validation Loss: 2.8603, Validation Accuracy: 0.2080
Epoch 6/20
Train Loss: 0.6641, Train Accuracy: 0.7496
Validation Loss: 3.1561, Validation Accuracy: 0.2880
Epoch 7/20
Train Loss: 0.6016, Train Accuracy: 0.7882
Validation Loss: 2.9842, Validation Accuracy: 0.2720
Epoch 8/20
Train Loss: 0.5517, Train Accuracy: 0.7984
Validation Loss: 2.8243, Validation Accuracy: 0.3080
Epoch 9/20
Train Loss: 0.4874, Train Accuracy: 0.8295
Validation Loss: 1.6595, Validation Accuracy: 0.4880
Epoch 10/20
Train Loss: 0.4564, Train

In [ ]:
# Plot Training Metrics
plt.figure(figsize=(12, 6))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("Loss over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Plot Validation Metrics
plt.figure(figsize=(12, 6))
plt.plot(train_accuracies, label="Train Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.title("Accuracy over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()